In [1]:
#Special module written for this class
#This provides access to data and to helper functions from previous weeks
import lucem_illud #pip install git+git://github.com/Computational-Content-Analysis-2018/lucem_illud.git

#All these packages need to be installed from pip
import requests #for http requests
import bs4 #called `beautifulsoup4`, an html parser
import pandas #gives us DataFrames
import docx #reading MS doc files, install as `python-docx`

#Stuff for pdfs
#Install as `pdfminer2`
import pdfminer.pdfinterp
import pdfminer.converter
import pdfminer.layout
import pdfminer.pdfpage

#These come with Python
import re #for regexs
import urllib.parse #For joining urls
import io #for making http requests look like files
import json #For Tumblr API responses
import os.path #For checking if files exist
import os #For making directories

In [2]:
wikipedia_base_url = 'https://en.wikipedia.org'
wikipedia_women_in_medicine = 'https://en.wikipedia.org/wiki/Women_in_medicine'
women_in_medicine_save = 'wikipedia_women_in_medicine.html'

In [3]:
requests.get(wikipedia_women_in_medicine)

<Response [200]>

In [4]:
wikiWiMRequest = requests.get(wikipedia_women_in_medicine)
print(wikiWiMRequest.text[:1000])

<!DOCTYPE html>
<html class="client-nojs" lang="en" dir="ltr">
<head>
<meta charset="UTF-8"/>
<title>Women in medicine - Wikipedia</title>
<script>document.documentElement.className = document.documentElement.className.replace( /(^|\s)client-nojs(\s|$)/, "$1client-js$2" );</script>
<script>(window.RLQ=window.RLQ||[]).push(function(){mw.config.set({"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"Women_in_medicine","wgTitle":"Women in medicine","wgCurRevisionId":822099984,"wgRevisionId":822099984,"wgArticleId":14457458,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Pages with disallowed DISPLAYTITLE modifications","CS1 Dutch-language sources (nl)","CS1 Norwegian Bokmål-language sources (nb)","CS1 Czech-language sources (cs)","CS1 Spanish-language sources (es)","Articles to be expanded from October 2011","All articles to be expanded","Articles using small message boxes","Vag

In [16]:
wikiWiMSoup = bs4.BeautifulSoup(wikiWiMRequest.text, 'html.parser')
print(wikiWiMSoup.text[:200])





Women in medicine - Wikipedia
document.documentElement.className = document.documentElement.className.replace( /(^|\s)client-nojs(\s|$)/, "$1client-js$2" );
(window.RLQ=window.RLQ||[]).push(functi


In [6]:
with open(women_in_medicine_save, mode='w', encoding='utf-8') as f:
    f.write(wikiWiMRequest.text)

In [7]:
contentPTags = wikiWiMSoup.body.findAll('p')
for pTag in contentPTags[:3]:
    print(pTag.text)

Historically and presently, in many parts of the world, women's participation in the profession of medicine (as physicians or surgeons for instance) has been significantly restricted. However, women's informal practice of medicine in roles such as caregivers or as allied health professionals has been widespread. Most countries of the world now provide women with equal access to medical education. However, not all countries ensure equal employment opportunities,[1] and gender equality has yet to be achieved within medical specialties and around the world,[2] despite studies suggest that female doctors may be providing higher-quality care than male doctors.[3][4]




In [8]:
contentParagraphs = []
for pTag in contentPTags:
    #strings starting with r are raw so their \'s are not modifier characters
    #If we didn't start with r the string would be: '\\[\\d+\\]'
    contentParagraphs.append(re.sub(r'\[\d+\]', '', pTag.text))

#convert to a DataFrame
contentParagraphsDF = pandas.DataFrame({'paragraph-text' : contentParagraphs})
print(contentParagraphsDF)

                                       paragraph-text
0   Historically and presently, in many parts of t...
1                                                    
2                                                    
3   In 1540, Henry VIII of England granted the cha...
4   At the beginning of the twenty-first century i...
5   However, the practice of medicine remains disp...
6   Moreover, there are skews within the medical p...
7   Women continue to dominate in nursing. In 2000...
8   Biomedical research and academic medical profe...
9   The glass ceiling is used as a metaphor to con...
10  One study surveyed physician mothers and their...
11  According to this study, one third of physicia...
12  This article provided an overview on the histo...
13  In an article titled "I'm too used to it”: A l...
14  Specifically, this article described how power...
15  On the topic of power dynamics, another study ...
16  Another study describes sexual harassment as a...
17  Women have successfully 

In [9]:
findNumber = r'\d'
regexResults = re.search(findNumber, 'not a number, not a number, numbers 2134567890, not a number')
regexResults

<_sre.SRE_Match object; span=(36, 37), match='2'>

In [10]:
print(regexResults.group(0))

2


In [11]:
findNumbers = r'\d+'
regexResults = re.search(findNumbers, 'not a number, not a number, numbers 2134567890, not a number')
print(regexResults.group(0))

2134567890


In [12]:
otherPAgeURLS = []
for paragraphNum, pTag in enumerate(contentPTags):
    tagLinks = pTag.findAll('a', href=re.compile('/wiki/'), class_=False)
    for aTag in tagLinks:
        relurl = aTag.get('href')
        linkText = aTag.text
        otherPAgeURLS.append((
            urllib.parse.urljoin(wikipedia_base_url, relurl),
            paragraphNum,
            linkText,
        ))
print(otherPAgeURLS[:10])

[('https://en.wikipedia.org/wiki/Physician', 0, 'physicians'), ('https://en.wikipedia.org/wiki/Allied_health_professions', 0, 'allied health professionals'), ('https://en.wikipedia.org/wiki/Medical_education', 0, 'medical education'), ('https://en.wikipedia.org/wiki/Specialty_(medicine)', 0, 'specialties'), ('https://en.wikipedia.org/wiki/Professionalization', 3, 'professionalizing'), ('https://en.wikipedia.org/wiki/Allied_health_professions', 3, 'allied health professions'), ('https://en.wikipedia.org/wiki/Nursing', 3, 'nursing'), ('https://en.wikipedia.org/wiki/Midwifery', 3, 'midwifery'), ('https://en.wikipedia.org/wiki/Medical_education', 3, 'medical education'), ('https://en.wikipedia.org/wiki/National_Institutes_of_Health', 8, 'National Institutes of Health')]


In [13]:
contentParagraphsDF['source'] = [wikipedia_women_in_medicine] * len(contentParagraphsDF['paragraph-text'])
contentParagraphsDF['paragraph-number'] = range(len(contentParagraphsDF['paragraph-text']))

contentParagraphsDF

,paragraph-text,source,paragraph-number
0,"Historically and presently, in many parts of t...",https://en.wikipedia.org/wiki/Women_in_medicine,0
1,,https://en.wikipedia.org/wiki/Women_in_medicine,1
2,,https://en.wikipedia.org/wiki/Women_in_medicine,2
3,"In 1540, Henry VIII of England granted the cha...",https://en.wikipedia.org/wiki/Women_in_medicine,3
4,At the beginning of the twenty-first century i...,https://en.wikipedia.org/wiki/Women_in_medicine,4
5,"However, the practice of medicine remains disp...",https://en.wikipedia.org/wiki/Women_in_medicine,5
6,"Moreover, there are skews within the medical p...",https://en.wikipedia.org/wiki/Women_in_medicine,6
7,Women continue to dominate in nursing. In 2000...,https://en.wikipedia.org/wiki/Women_in_medicine,7
8,Biomedical research and academic medical profe...,https://en.wikipedia.org/wiki/Women_in_medicine,8
9,The glass ceiling is used as a metaphor to con...,https://en.wikipedia.org/wiki/Women_in_medicine,9


In [14]:
contentParagraphsDF['source-paragraph-number'] = [None] * len(contentParagraphsDF['paragraph-text'])
contentParagraphsDF['source-paragraph-text'] = [None] * len(contentParagraphsDF['paragraph-text'])

def getTextFromWikiPage(targetURL, sourceParNum, sourceText):
    parsDict = {'source' : [], 'paragraph-number' : [], 'paragraph-text' : [], 'source-paragraph-number' : [],  'source-paragraph-text' : []}
    r = requests.get(targetURL)
    soup = bs4.BeautifulSoup(r.text, 'html.parser')
    for parNum, pTag in enumerate(soup.body.findAll('p')):
        parsDict['paragraph-text'].append(re.sub(r'\[\d+\]', '', pTag.text))
        parsDict['paragraph-number'].append(parNum)
        parsDict['source'].append(targetURL)
        parsDict['source-paragraph-number'].append(sourceParNum)
        parsDict['source-paragraph-text'].append(sourceText)
    return pandas.DataFrame(parsDict)

In [15]:
for urlTuple in otherPAgeURLS[:3]:
    contentParagraphsDF = contentParagraphsDF.append(getTextFromWikiPage(*urlTuple),ignore_index=True)
contentParagraphsDF

,paragraph-number,paragraph-text,source,source-paragraph-number,source-paragraph-text
0,0,"Historically and presently, in many parts of t...",https://en.wikipedia.org/wiki/Women_in_medicine,None,None
1,1,,https://en.wikipedia.org/wiki/Women_in_medicine,None,None
2,2,,https://en.wikipedia.org/wiki/Women_in_medicine,None,None
3,3,"In 1540, Henry VIII of England granted the cha...",https://en.wikipedia.org/wiki/Women_in_medicine,None,None
4,4,At the beginning of the twenty-first century i...,https://en.wikipedia.org/wiki/Women_in_medicine,None,None
5,5,"However, the practice of medicine remains disp...",https://en.wikipedia.org/wiki/Women_in_medicine,None,None
6,6,"Moreover, there are skews within the medical p...",https://en.wikipedia.org/wiki/Women_in_medicine,None,None
7,7,Women continue to dominate in nursing. In 2000...,https://en.wikipedia.org/wiki/Women_in_medicine,None,None
8,8,Biomedical research and academic medical profe...,https://en.wikipedia.org/wiki/Women_in_medicine,None,None
9,9,The glass ceiling is used as a metaphor to con...,https://en.wikipedia.org/wiki/Women_in_medicine,None,None
